# Adapt JFnet to healthy vs. diseased decision on Kaggle DR data

### Motivation

* optretina: label noise & transfer of JFnet to new dataset with potentially differing statistics
* kaggle dr: this should give us a feeling on how good we can get roughly

In [ ]:
import sys
sys.path.append('..')
import os

import theano
import theano.tensor as T
import lasagne
import keras
from keras.utils import np_utils

import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

from progressbar import ProgressBar
import models
from datasets import KaggleDR
# import plotting
# plotting.allow_plot()
import seaborn as sns
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
# %matplotlib notebook

## Image preprocessing

In [ ]:
# Training images
%run ../convert_JF.py --directory=../data/kaggle_dr/train --convert_directory=../data/kaggle_dr/train_JF_512 --crop_size=512 --extension=jpeg --n_proc=4

# Test images
# TODO: I don't have the raw test images get the resized ones from vaneeda
%run ../convert_JF.py --directory=../data/kaggle_dr/test --convert_directory=../data/kaggle_dr/test_JF_512 --crop_size=512 --extension=jpeg --n_proc=4

## Feature extraction

from Jeffrey de Fauw's network, originally trained on Kaggle DR competition

In [ ]:
def extract_features(source_path=None, source_filenames=None, last_layer=None, outfile=None, batch_size=2):
    input_var = T.tensor4('inputs')
    weights = '../models/jeffrey_df/2015_07_17_123003_PARAMSDUMP.pkl'
    network = models.jeffrey_df(input_var=input_var, width=512, height=512, filename=weights)
    output_layer = network[last_layer]

    feature_activations = lasagne.layers.get_output(output_layer)
    forward_pass = theano.function([input_var], feature_activations)

    dataset = KaggleDR(path_data=source_path, filename_targets=source_filenames,
                       preprocessing=KaggleDR.jf_trafo)

    n_features = np.prod(output_layer.output_shape[1:])
    if os.path.exists(outfile):
        print 'Feature file already exists, aborting.'
        return

    fp = np.memmap(outfile, dtype=theano.config.floatX, mode='w+',
                   shape=(dataset.n_samples,) + output_layer.output_shape[1:]) # Each sample might still be of dim > 1

    n_batches = np.ceil(dataset.n_samples/batch_size)

    p = ProgressBar(n_batches)
    
    for i, batch in enumerate(dataset.iterate_minibatches(np.arange(dataset.n_samples), batch_size)):
        p.animate(i)
        inputs, _ = batch
        fp[i*batch_size:min((i+1)*batch_size, dataset.n_samples)] = forward_pass(inputs)
        
    del fp # close memory mapped array
    
    return

In [ ]:
last_layer = 'conv_combined'

In [ ]:
# Training images
extract_features(source_path='../data/kaggle_dr/train_JF_512', 
                 source_filenames='../data/kaggle_dr/trainLabels_bin.csv', 
                 last_layer=last_layer, 
                 outfile='../data/kaggle_dr/feat_train_' + last_layer + '.npy',
                 batch_size=2)

In [ ]:
# Test images
extract_features(source_path='../data/kaggle_dr/test_JF_512', 
                 source_filenames='../data/kaggle_dr/retinopathy_solution_bin.csv', 
                 last_layer=last_layer, 
                 outfile='../data/kaggle_dr/feat_test_' + last_layer + '.npy',
                 batch_size=2)

## Train classifier on fixed JFnet features

### Load data

#### Kaggle DR

In [ ]:
def load_kaggle_dr_features_and_targets():
    
    # Training data
    y = pd.read_csv('../data/kaggle_dr/trainLabels_bin.csv').level.values
    nb_samples = len(y)
    X = np.memmap('../data/kaggle_dr/feat_train_conv_combined.npy', dtype=theano.config.floatX, mode='r', 
                        shape=(nb_samples, 768))

    _, nb_features = X.shape

    print 'nb_samples for training:', nb_samples, 'nb_features:', nb_features

    # Test data
    df_test = pd.read_csv('../data/kaggle_dr/retinopathy_solution_bin.csv')
    y_test = df_test.level.values

    nb_samples = len(y_test)

    X_test = np.memmap('../data/kaggle_dr/feat_test_conv_combined.npy', dtype=theano.config.floatX, mode='r', 
                       shape=(nb_samples, 768))

    _, nb_features = X_test.shape

    print 'nb_samples for testing:', nb_samples, 'nb_features:', nb_features
    
    return X, y, X_test, y_test, df_test

#### Optretina

In [ ]:
def include_indices(exclude_path, df):
    exclude_filenames = [fn.split('.')[0] for fn in os.listdir(exclude_path)]

    images = df.filename.apply(lambda fn: fn.split('.')[0]).values
    centre_ids = df.centre_id.values.astype(str)
    filenames = pd.Series(['_'.join(centre_id_and_image) for
                           centre_id_and_image in
                           zip(centre_ids, images)])

    return filenames.index[~filenames.isin(exclude_filenames)]

def load_optretina_features_and_targets():
    exclude_path = '../data/optretina/data_JF_512_exclude'
    df = pd.read_csv('../data/optretina/OR_diseased_labels.csv')
    include_idx = include_indices(exclude_path, df)
    n_samples = len(df)
    X = np.memmap('../data/optretina/feat_act_JFnet_18.npy', dtype=theano.config.floatX, mode='r', 
                       shape=(n_samples, 256*7*7))

    X = X[include_idx]
    y = df.diseased.values[include_idx]
    n_images_total = len(df)
    df = df.iloc[include_idx]
    n_images_accepted = len(df)

    print('Excluding {} out of {} images.'.format(n_images_total -
                                                  n_images_accepted,
                                                  n_images_total))
    print('Relative class frequencies: ', pd.value_counts(y)/float(len(y)))
    
    X, X_test, y, y_test, df, df_test = train_test_split(X, y, df, stratify=y, test_size=0.2, random_state=1234)
    
    return X, y, X_test, y_test, df_test

In [ ]:
X, y, X_test, y_test, df_test = load_kaggle_dr_features_and_targets()

### Network definition

Sanity check that model has enough capacity by overfitting the training data without regularization revealed:
* Linear classifier on top of layer 18 features (flattened) did not achieve training accuracy within a reasonable time
* Single hidden layer with 1024 units: train_acc = 0.999 after 70 epochs, 0.9999 after 85 ep.
* Two hidden layers (0:1024, 1:512 units): train_acc =  0.9998 after 39 ep.

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2

l2_lambda = 0.001
nb_features = 768  # 256*7*7
nb_classes = 2

# Two hidden layers
mlp = Sequential()
fc1 = Dense(1024, input_dim=nb_features, init='glorot_normal',
                activation=LeakyReLU(alpha=0.3),
                W_regularizer=l2(l2_lambda))
fc1.name = 'fc1'
mlp.add(fc1)
fc2 = Dense(512, init='glorot_normal',
                activation=LeakyReLU(alpha=0.3),
                W_regularizer=l2(l2_lambda))
fc2.name = 'fc2'
mlp.add(fc2)
prob = Dense(nb_classes,
                init='glorot_normal',
                activation='softmax',
                W_regularizer=l2(l2_lambda))
prob.name = 'prob'
mlp.add(prob)

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
# optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True)
mlp.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
mlp.summary()

### Network training

In [ ]:
%matplotlib notebook

from util import AdaptiveLearningRateScheduler
from util import TrainingMonitor

Y = np_utils.to_categorical(y)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, stratify=y, test_size=0.2, random_state=1234)


callbacks = []
history = keras.callbacks.History()
callbacks.append(history)
callbacks.append(TrainingMonitor(history))
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               patience=10,
                                               verbose=1,
                                               mode='auto')
callbacks.append(early_stopping)
callbacks.append(keras.callbacks.ModelCheckpoint('mlp_best_weights.h5', 
                                monitor='val_loss', 
                                verbose=0, 
                                save_best_only=True, 
                                mode='auto'))


initial_lr = float(optimizer.lr.get_value())
callbacks.append(AdaptiveLearningRateScheduler(initial_lr=initial_lr,
                                               decay=0.5,
                                               patience=20,
                                               verbose=1))

cross_entropy_train  = mlp.evaluate(X_train, Y_train)
cross_entropy_chance = - np.log(1.0/nb_classes)
print('Cross entropy for chance level:', cross_entropy_chance)
print('Cross entropy before training:', cross_entropy_train)

mlp.fit(X_train, Y_train, nb_epoch=1000, batch_size=256,
          class_weight={0:1, 1:1}, validation_data=(X_val, Y_val),
          show_accuracy=True, verbose=1, callbacks=callbacks)

In [ ]:
plotting.close_all()

## Analyse performance

### Load test data

### Load weights

In [ ]:
mlp.load_weights('mlp_best_weights.h5')

### Perform predictions

In [ ]:
posteriors = mlp.predict_proba(X_test)
HEALTHY, DISEASED = 0, 1
df_test.loc[:, 'prob_healthy'] = posteriors[:, HEALTHY]
df_test.loc[:, 'prob_diseased'] = posteriors[:, DISEASED]
df_test.loc[:,'pred'] = mlp.predict_classes(X_test)
df_test.to_csv('test_predictions.csv', index=False)

### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, df_test.pred)

### ROC curve with DISEASED as "positive" class

In [ ]:
from sklearn.metrics import roc_curve, auc
f_diseased_r, t_diseased_r, thresholds = roc_curve(y_test,
                                                   posteriors[:, DISEASED],
                                                   pos_label=DISEASED)

roc_auc = auc(f_diseased_r, t_diseased_r, reorder=True)

import matplotlib.pyplot as plt
# %matplotlib inline
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})


plt.figure()
plt.plot(f_diseased_r, t_diseased_r,
         label='ROC curve (auc = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.scatter([0.05], [0.8], color='g', s=50, label='recommendation British Diabetic Association')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('false diseased rate (1 - specificity)')
plt.ylabel('true diseased rate (sensitivity)')
plt.legend(loc="lower right")
plt.show()

### Precision-Recall curve with DISEASED as "positive" class

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, posteriors[:,
                                                        DISEASED],
                                              pos_label=DISEASED)
pr_auc = auc(recall, precision, reorder=False)
# Plot Precision-Recall curve
plt.figure()
plt.plot(recall, precision, label='PR curve (auc= %0.2f)' % pr_auc)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision recall curve')
plt.legend(loc="lower left")
plt.show()

In [ ]:
del X_test

## Question: Could we improve the performance further by fine-tuning the entire network, i.e. jfnet layers 1 to 18 together with the MLP from here?

### Regarding the implementation, we have two options:
1. Implement jfnet layers 1 to 18 in keras; that way we could use all the convenient training code from keras. Unfortunately however, keras does not have the concept of untied biases (maybe we could take the spatial mean over the biases as the std/mean is around 0.003 for all the layers)

2. Export MLP obtained here as lasagne model. For now this seems to be the faster option. Vaneeda is anyway used to her training code to be used on cantor

## Option 1

In [ ]:
jfnet18 = models.jfnet18_to_keras(filename='../models/jeffrey_df/2015_07_17_123003_PARAMSDUMP.pkl')

### Extract JFnet18 features (keras implementation)

In [ ]:
from keras.layers.core import Flatten
jfnet18.add(Flatten())
jfnet18.compile(optimizer='sgd', loss='binary_crossentropy')
jfnet18.summary()

In [ ]:
batch_size = 4
dataset = KaggleDR(path_data='../data/kaggle_dr/train_JF_512', 
                   filename_targets='../data/kaggle_dr/trainLabels_bin.csv',
                   preprocessing=KaggleDR.jf_trafo)

feature_file = '../data/kaggle_dr/feat_train_jf18_keras.npy'
if not os.path.exists(feature_file):
    fp = np.memmap(feature_file, dtype=theano.config.floatX, mode='w+',
                   shape=(dataset.n_samples, 256*7*7))

    n_batches = np.ceil(dataset.n_samples/batch_size)
    p = ProgressBar(n_batches)

    for i, batch in enumerate(dataset.iterate_minibatches(np.arange(dataset.n_samples), batch_size, shuffle=False)):
        p.animate(i)
        inputs, _ = batch
        fp[i*batch_size:min((i+1)*batch_size, dataset.n_samples)] = jfnet18.predict(inputs, batch_size=4, verbose=0)

    del fp # close memory mapped array
else:
    print 'Feature file already exists, aborting.'

#### Stack jfnet18 and mlp

In [ ]:
from keras.layers.core import Flatten
model = Sequential()
for layer in jfnet18.layers:
    layer.initial_weights = None # This is a current workaround; I do not understand why 'add' wants to set the weights of the previous layer?! - but I tested that model has the same weights as jfnet18 (up to incl. layer 18)
    model.add(layer)
    
# Flatten output of layer 18 (256, 7, 7) to 12544 features:
model.add(Flatten())

for layer in mlp.layers:
    model.add(layer)
    
model.summary()

In [ ]:
# optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
optimizer = keras.optimizers.SGD(lr=0.00001, momentum=0.9, decay=0.0, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=optimizer)

### Fine-tune jfnet18 together with MLP

In [ ]:
dataset = KaggleDR(path_data='../data/kaggle_dr/train_JF_512', 
                   filename_targets='../data/kaggle_dr/trainLabels_bin.csv',
                   preprocessing=KaggleDR.jf_trafo)

In [ ]:
n_samples = dataset.n_samples
from sklearn.cross_validation import train_test_split
idx_train, idx_val = train_test_split(np.arange(n_samples), test_size=0.2, stratify=dataset.y[:n_samples])

In [ ]:
%matplotlib notebook

nb_epoch = 1000
batch_size = 4
nb_classes = 2

from util import AdaptiveLearningRateScheduler
from util import TrainingMonitor
from keras.utils.generic_utils import Progbar

callbacks = []
history = keras.callbacks.History()
callbacks.append(history)
callbacks.append(keras.callbacks.BaseLogger())
callbacks.append(TrainingMonitor(history))
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               patience=30,
                                               verbose=1,
                                               mode='auto')
callbacks.append(early_stopping)

# initial_lr = float(optimizer.lr.get_value())
initial_lr = 1e-6
callbacks.append(AdaptiveLearningRateScheduler(initial_lr=initial_lr,
                                               decay=0.5,
                                               patience=3,
                                               verbose=1))

callbacks = keras.callbacks.CallbackList(callbacks)

callbacks._set_model(model)
callbacks._set_params({
    'batch_size': batch_size,
    'nb_epoch': nb_epoch,
    'nb_sample': len(idx_train),
    'verbose': 1,
    'do_validation': True,
    'metrics': ['loss', 'acc', 'val_loss', 'val_acc'],
})
callbacks.on_train_begin()

print('Train on %d samples, validate on %d samples' % (len(idx_train), len(idx_val)))

model.stop_training = False
epoch_logs = {}

for epoch in range(nb_epoch):
    callbacks.on_epoch_begin(epoch, epoch_logs)
    # Training
    for batch_index, (X, y) in enumerate(dataset.iterate_minibatches(idx_train, batch_size, shuffle=True)):
        batch_logs = {}
        batch_logs['batch'] = batch_index
        batch_logs['size'] = len(y)
        callbacks.on_batch_begin(batch_index, batch_logs)
        Y = np_utils.to_categorical(y, nb_classes=nb_classes)
        batch_logs['loss'], batch_logs['acc'] = model.train_on_batch(X, Y, accuracy=True, class_weight={0:1, 1:2.77})
        callbacks.on_batch_end(batch_index, batch_logs)

    # Validation
    val_loss = np.zeros((len(idx_val),1))
    val_acc = np.zeros((len(idx_val),1))
    progbar = Progbar(target=len(idx_val))
    for i, (X, y) in enumerate(dataset.iterate_minibatches(idx_val, batch_size, shuffle=False)):
        Y = np_utils.to_categorical(y, nb_classes=nb_classes)
        vl, va = model.test_on_batch(X, Y, accuracy=True)
        val_loss[i*batch_size:min((i+1)*batch_size, len(idx_val))] = vl
        val_acc[i*batch_size:min((i+1)*batch_size, len(idx_val))] = va
        progbar.update(min(i+batch_size, len(idx_val)))
        
    epoch_logs['val_loss'], epoch_logs['val_acc'] = np.mean(val_loss), np.mean(val_acc)
    
    callbacks.on_epoch_end(epoch, epoch_logs)
    if model.stop_training:
        break

callbacks.on_train_end()

In [ ]:
model.save_weights('jf18mlp.h5')

In [ ]:
model.load_weights('jf18mlp.h5')

### Perform predictions, iterating over the test set in batches

In [ ]:
from keras.utils.generic_utils import Progbar

test_data = KaggleDR(path_data='../data/kaggle_dr/test_JF_512', 
                   filename_targets='../data/kaggle_dr/retinopathy_solution_bin.csv',
                   preprocessing=KaggleDR.jf_trafo)
batch_size = 2
print('Test on %d samples...' % (test_data.n_samples))

posteriors = np.zeros((test_data.n_samples, 2))
progbar = Progbar(target=test_data.n_samples)
for i, (X, _) in enumerate(test_data.iterate_minibatches(np.arange(test_data.n_samples), batch_size, shuffle=False)):
    posteriors[i*batch_size:min((i+1)*batch_size, test_data.n_samples)] = model.predict_proba(X, verbose=0)
    progbar.update(min(i+batch_size, test_data.n_samples))

## Option 2

### Convert parameter (here, weights and biases are meant) layout of keras to be usable by lasagne

lasagne needs a list of numpy arrays with alternating weights and biases: [W, b, W, b, W, b].

In [ ]:
param_values = [layer_params.get_value() for layer_params in mlp.get_params()[0]]
[layer_param.shape for layer_param in param_values]

### Dump weights to be used with lasagne later on:

In [ ]:
np.savez('weights.npz', *param_values)

### Demo of MLP in lasagne (to be used as part of code for fine-tuning JFnet 1 to 18 together with MLP)

#### Network definition
optionally add dropout layers

In [ ]:
import lasagne
from lasagne.layers import InputLayer, DenseLayer, NonlinearityLayer
from lasagne.nonlinearities import softmax, LeakyRectify

# Two hidden layer MLP in keras:
# mlp = Sequential()
# mlp.add(Dense(1024, input_dim=nb_features, init='glorot_normal',
#                 activation=LeakyReLU(alpha=0.3),
#                 W_regularizer=l2(l2_lambda)))
# mlp.add(Dense(512, init='glorot_normal',
#                 activation=LeakyReLU(alpha=0.3),
#                 W_regularizer=l2(l2_lambda)))
# mlp.add(Dense(nb_classes,
#                 init='glorot_normal',
#                 activation='softmax',
#                 W_regularizer=l2(l2_lambda)))

mlp = {}
mlp['input'] = InputLayer((None, nb_features))
mlp['fc1'] = DenseLayer(mlp['input'], num_units=1024)
mlp['fc2'] = DenseLayer(mlp['fc1'], num_units=512, nonlinearity=LeakyRectify(leakiness=0.3))
mlp['fc3'] = DenseLayer(mlp['fc2'], num_units=nb_classes, nonlinearity=LeakyRectify(leakiness=0.3))
mlp['prob'] = NonlinearityLayer(mlp['fc3'], softmax)

#### Load and set MLP weights

In [ ]:
with np.load('weights.npz') as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]
    lasagne.layers.set_all_param_values(mlp['prob'], param_values)